<a href="https://colab.research.google.com/github/edwin-yan/Passion-Fruit-Disease-Detection/blob/andrew/inference_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Look at `Inference` section: 
https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb

And this file:
https://github.com/ultralytics/yolov5/blob/master/detect.py


# SET UP
## Mounting

Note that this assumes that your google drive folder has the same document layout. You must modify if that is not the case.


From your root Google Drive directory assumes there are these folders:
* `JHU/DNN/FinalProject/best.pt` -- weights
* `JHU/DNN/FinalProject/Test_Images` -- test images

with Test images unzipped in those directories



# **TODO:** GLOBAL VARIABLE set to `True` if using colab and `False` if not!

In [1]:
IS_COLAB = True 

# **TODO:** CHANGE `IMG_PATH` if your google drive directory is different

In [2]:
if IS_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  IMG_PATH = '/content/drive/MyDrive/JHU/DNN/FinalProject'
else:
  IMG_PATH= '.'

Mounted at /content/drive


In [3]:
TEST_IMAGES = f'{IMG_PATH}/Test_Images'

In [4]:
# WEIGHTS = f'{IMG_PATH}/best.pt'

In [5]:
# WEIGHTS = f'{IMG_PATH}/best_20211105_Dina.pt'

In [6]:
# WEIGHTS = f'{IMG_PATH}/best_edwin_20211107.pt'

In [7]:
WEIGHTS = f'{IMG_PATH}/best_andrew_20211108_yolo5s_150epochs_batch32.pt' 

Start timer

In [8]:
import time

t0 = time.time()

Inference

https://github.com/ultralytics/yolov5#quick-start-examples

In [9]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 10147, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 10147 (delta 50), reused 73 (delta 42), pack-reused 10054
Receiving objects: 100% (10147/10147), 10.36 MiB | 16.45 MiB/s, done.
Resolving deltas: 100% (7031/7031), done.
/content/yolov5
     |████████████████████████████████| 596 kB 4.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Perform inference

```
! python detect.py --source {TEST_IMAGES} --weights {WEIGHTS} --save-txt --save-conf --conf-thres 0 --max-det 20
```

Try default 1000 predictions

In [10]:
! python detect.py --source {TEST_IMAGES} --weights {WEIGHTS} --save-txt --save-conf --conf-thres 0

detect: weights=['/content/drive/MyDrive/JHU/DNN/FinalProject/best_andrew_20211108_yolo5s_150epochs_batch32.pt'], source=/content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images, imgsz=[640, 640], conf_thres=0.0, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-76-g79bca2b torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/931 /content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images/ID_0082Y2NI.jpg: 640x640 851 fruit_woodinesss, 149 fruit_brownspots, Done. (0.007s)
image 2/931 /content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images/ID_00DCPV52.jpg: 640x640 17 fruit_healthys, 273 fruit_woodinesss, 710 fru

# TODO: Adjust files to Zindi format

.txt files are in the format of:

.txt per image: 

/content/yolov5/runs/detect/exp/labels

In [11]:
category_index = {0: {'id': 0, 'name': 'fruit_healthy'},
                1: {'id': 1, 'name': 'fruit_woodiness'},
                 2: {'id': 3, 'name': 'fruit_brownspot'}}

TODO: If images do not have score, add blank series with ID only

In [12]:

import os
import pandas as pd
path = '/content/yolov5/runs/detect/exp/labels/'
filelist = os.listdir(path) 
df_write = pd.DataFrame(columns = ['Image_ID', 'class', 'confidence', 'ymin','xmin','ymax','xmax'])

img_size = 512

for file in filelist:
  data = pd.read_csv(path+file, sep=" ", header=None)
  data = data.rename(columns={0:'class',1:'x',2:'y',3:'w',4:'h',5:'confidence'})
  data['class'] = data['class'].apply(lambda x: category_index[x]['name'])
  data['ymin'] = (data['y']-data['h']/2)*img_size
  data['xmin'] = (data['x']-data['w']/2)*img_size
  data['ymax'] = (data['y']+data['h']/2)*img_size
  data['xmax'] = (data['x']+data['w']/2)*img_size
  data['Image_ID'] = file.split('.')[0]

  cols = ['Image_ID','class','confidence','ymin',	'xmin',	'ymax',	'xmax']
  data = data[cols]

  df_write = df_write.append(data.nlargest(4, 'confidence'))


In [13]:
df_write

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
999,ID_NE7KN6UJ,fruit_healthy,0.951201,181.000192,160.999936,344.999936,381.000192
998,ID_NE7KN6UJ,fruit_healthy,0.045437,0.000000,109.999744,19.000013,153.999744
997,ID_NE7KN6UJ,fruit_brownspot,0.005105,237.999872,0.000026,396.999936,41.000013
996,ID_NE7KN6UJ,fruit_brownspot,0.001260,312.000256,252.999936,512.000256,396.000000
999,ID_SNM6060U,fruit_brownspot,0.831851,109.999872,218.999808,343.999744,403.999744
...,...,...,...,...,...,...,...
996,ID_B86CXFQ9,fruit_woodiness,0.000452,110.999808,380.000000,332.000000,510.999808
999,ID_A7GQPXFU,fruit_brownspot,0.804579,220.000000,259.000064,398.999808,400.000256
998,ID_A7GQPXFU,fruit_brownspot,0.668539,87.000064,330.999808,231.000064,451.999744
997,ID_A7GQPXFU,fruit_brownspot,0.035988,355.000064,0.000256,512.000256,120.000256


See which images have no results:
https://www.statology.org/pandas-unique-values-in-column/

In [14]:
labeled_set_imgs = set(df_write.Image_ID.unique())

Create array of all test images

In [15]:
TEST_IMAGES

'/content/drive/MyDrive/JHU/DNN/FinalProject/Test_Images'

In [16]:
list_of_imgs = []
for file in os.listdir(TEST_IMAGES):
  list_of_imgs.append(file.split('.')[0])

In [17]:
import numpy as np

In [18]:
all_set_imgs = set(list_of_imgs)

# Images that do not have confidence above 0.5:

https://stackoverflow.com/questions/3462143/get-difference-between-two-lists

In [19]:
arr_no_labels = np.array(list(all_set_imgs - labeled_set_imgs))

In [20]:
arr_no_labels

array([], dtype=float64)

In [21]:
arr_no_labels.size

0

In [22]:
for a in arr_no_labels:
  data['Image_ID'] = a
  df_write = df_write.append(data)

In [23]:
labeled_set_imgs = set(df_write.Image_ID.unique())

In [24]:
df_write.to_csv('/content/drive/MyDrive/JHU/DNN/FinalProject/submission_yolo_unnormalized_top4.csv', encoding='utf-8', index=False)

In [25]:
tend = time.time()
print('Elapsed time: ', (tend-t0)/60, ' minutes')

Elapsed time:  22.497230883439382  minutes
